In [1]:
import geemap
import ee

Map = geemap.Map(center=[30, 112])
Map

Map(center=[30, 112], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [2]:
import os
os.chdir(r'D:\Data\parper\data')

In [3]:
shp = geemap.shp_to_ee('shp/JX.shp')
roi = shp.geometry()
Map.center_object(roi, 8)
Map.addLayer(roi, {}, 'roi')

In [4]:
points = geemap.shp_to_ee('shp/landuse_point.shp')

Map.addLayer(points, {'color': 'FF0000'}, 'points')

In [5]:
visParams = {'max': 3000, 'min': 0, 'bands': ['B4', 'B3', 'B2']}

In [17]:
Map.addLayer(s2_year[0], visParams, 's2')

In [22]:
from Monarch import user_gee
year = 2018
s2_year = []
lswis = []
ndvis = []
evis = []
for i in range(1, 13):
    s2 = ee.ImageCollection("COPERNICUS/S2")\
            .filterBounds(roi)\
            .sort('CLOUD_COVERAGE_ASSESSMENT', False)\
            .filterDate(str(year), str(year+1))\
            .filter(ee.Filter.calendarRange(i, i, 'month'))
    s2_rm_cloud = s2.map(user_gee.rm_cloud_s2_sr)
    s2_z = s2_rm_cloud.merge(s2).median().select(['B2', 'B3', 'B4', 'B5', 'B8', 'B11'])
    a = s2.aggregate_array('system:time_start').map(lambda i: ee.Date(i).format('YYYY-MM-dd'))
    evi = s2_z.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': s2_z.select('B8'),
        'RED': s2_z.select('B4'),
        'BLUE': s2_z.select('B2')
    }).rename("S2_EVI")
    ndvi = s2_z.normalizedDifference(['B8', 'B4']).rename("S2_NDVI")
    lswi = s2_z.normalizedDifference(['B8', 'B11']).rename("S2_LSWI")
    ndvis.append(ndvi)
    lswis.append(lswi)
    evis.append(evi)
    s2_image = s2_z.addBands(ndvi)
    s2_image = s2_image.addBands(lswi)
    # s2_image = s2_image.addBands(evi)
    s2_year.append(s2_image.clip(roi))

In [23]:
image = ee.ImageCollection(s2_year)
geemap.download_ee_video(image, visParams, f'gif_data/S2_{year}.gif')

Generating URL...
Please wait ...
The GIF image has been saved to: D:\Data\parper\data\gif_data\S2_2018.gif


In [24]:
geemap.add_text_to_gif(f'gif_data/S2_{year}.gif', f'gif_data/S2_{year}_text.gif', xy=('3%', '5%'), text_sequence=[f'{year}-{i:02d}' for i in range(1, 13)], 
                       font_size=20, font_color='#ffffff', duration=1000)

In [25]:
geemap.show_image(f'gif_data/S2_{year}_text.gif')

Output()